<a href="https://colab.research.google.com/github/delhian/try_pytorch/blob/main/Torch_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Disign model (input, output size,)

2) Construct loss and optimizer

3) Training loop

-- forward pass: compute prediction and loss

-- backward pass: calculate gradients

-- update weights

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
X_numpy, y_numpy = datasets.make_classification(n_classes=2, n_samples=300, n_features=2, n_redundant=0, n_informative=2)
print(X_numpy.shape, y_numpy.shape)

(300, 2) (300,)


In [ ]:
X_numpy_train, X_numpy_test, y_numpy_train, y_numpy_test = train_test_split(X_numpy, y_numpy)
ss = StandardScaler()
X_numpy_train = ss.fit_transform(X_numpy_train)
X_numpy_test = ss.transform(X_numpy_test)
print(X_numpy_train.shape, X_numpy_test.shape, y_numpy_train.shape, y_numpy_test.shape)

(225, 2) (75, 2) (225,) (75,)


In [ ]:
X_train = torch.from_numpy(X_numpy_train.astype(np.float32))
X_test = torch.from_numpy(X_numpy_test.astype(np.float32))
Y_train = torch.from_numpy(y_numpy_train.astype(np.float32)).unsqueeze(-1)
Y_test = torch.from_numpy(y_numpy_test.astype(np.float32)).unsqueeze(-1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

torch.Size([225, 2]) torch.Size([75, 2]) torch.Size([225, 1]) torch.Size([75, 1])


In [ ]:
n_samples, n_features = X_train.shape
print(f'n_samples: {n_samples},\nn_features: {n_features}')


n_samples: 225,
n_features: 2


In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super().__init__()
    self.linear = nn.Linear(n_input_features, 1)
  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

In [ ]:
model = LogisticRegression(n_features)

In [ ]:
loss = nn.BCELoss()
learning_rate = 1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
epoches = 100
for epoch in range(epoches):
  Y_predicted = model(X_train)
  l = loss(Y_predicted, Y_train)
  l.backward()
  optimizer.step()
  optimizer.zero_grad()
  if epoch%10 == 0:
    [w, b] = model.parameters()
    print(f'epoch:{epoch + 1} loss: {l:.8f}')

epoch:1 loss: 0.52369803
epoch:11 loss: 0.34821492
epoch:21 loss: 0.32609686
epoch:31 loss: 0.31836903
epoch:41 loss: 0.31491071
epoch:51 loss: 0.31316996
epoch:61 loss: 0.31223109
epoch:71 loss: 0.31170148
epoch:81 loss: 0.31139335
epoch:91 loss: 0.31121013


In [ ]:
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted = y_predicted.round()
  acc = y_predicted.eq(Y_test).sum()/float(Y_test.shape[0])
  print(acc)

tensor(0.8667)


In [ ]:
with torch.no_grad():
  y_predicted = model(X_train).round()
  acc = y_predicted.eq(Y_train).sum()/float(Y_train.shape[0])
  print(acc)

tensor(0.8756)
